<a href="https://colab.research.google.com/github/Sidharth1999/Capstone-3/blob/main/Image_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#Imports
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import pathlib
import imageio
import functools

In [4]:
dataDir = "/content/drive/My Drive/Springboard-Capstone-3/data"
projectDir = "/content/drive/My Drive/Springboard-Capstone-3"

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sidharthr1999","key":"b06c0eb687ec8653837b4badf109296d"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Verify that kaggle commands work
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              229KB  2021-06-01 11:18:46           6831  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           4086  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1364  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27           2128  
mathurinache/twitter-edge-nodes                             Twitter Edge Nod

In [ ]:
dataset1 = "iamsouravbanerjee/indian-food-images-dataset"

In [ ]:
#Download dataset

!kaggle datasets download $dataset1 -p "$dataDir" --unzip

 97% 343M/354M [00:02<00:00, 128MB/s]
100% 354M/354M [00:03<00:00, 123MB/s]


In [ ]:
######## Preparing data for CNN models #########

In [4]:
#Obtain all 80 categories
categories = []
for file in os.listdir(dataDir): categories.append(file)
print(categories)

['adhirasam', 'aloo_gobi', 'aloo_matar', 'aloo_methi', 'aloo_shimla_mirch', 'aloo_tikki', 'anarsa', 'ariselu', 'bandar_laddu', 'basundi', 'bhatura', 'bhindi_masala', 'biryani', 'boondi', 'butter_chicken', 'chak_hao_kheer', 'cham_cham', 'chana_masala', 'chapati', 'chhena_kheeri', 'chicken_razala', 'chicken_tikka', 'chicken_tikka_masala', 'chikki', 'daal_baati_churma', 'daal_puri', 'dal_makhani', 'dal_tadka', 'dharwad_pedha', 'doodhpak', 'double_ka_meetha', 'dum_aloo', 'gajar_ka_halwa', 'gavvalu', 'ghevar', 'gulab_jamun', 'imarti', 'jalebi', 'kachori', 'kadai_paneer', 'kadhi_pakoda', 'kajjikaya', 'kakinada_khaja', 'kalakand', 'karela_bharta', 'kofta', 'kuzhi_paniyaram', 'lassi', 'ledikeni', 'litti_chokha', 'lyangcha', 'maach_jhol', 'makki_di_roti_sarson_da_saag', 'malapua', 'misi_roti', 'misti_doi', 'modak', 'mysore_pak', 'naan', 'navrattan_korma', 'palak_paneer', 'paneer_butter_masala', 'phirni', 'pithe', 'poha', 'poornalu', 'pootharekulu', 'qubani_ka_meetha', 'rabri', 'ras_malai', 'ras

In [5]:
#Extract all 4000 image paths for each of the 80 categories
def imagePathsFromCategory(category):
  generator = pathlib.Path(dataDir + '/' + category).glob('*.jpg')
  sorted_paths = sorted([x for x in generator])
  return sorted_paths 

image_paths = [imagePathsFromCategory(category) for category in categories]

#Confirm all 4000 image paths were extracted:
print(functools.reduce(lambda a, b: a+b, [len(categoryPaths) for categoryPaths in image_paths]))

4000


In [8]:
#Split each of the 80 categories into train, validation, and test
from sklearn.model_selection import train_test_split
train = []
validation = []
test = []
for i in range(len(image_paths)):
  train_images, test_images, _, _ = train_test_split(image_paths[i], range(len(image_paths[i])), test_size=0.20, random_state=42)
  train_images, validation_images, _, _ = train_test_split(train_images, range(len(train_images)), test_size=0.20, random_state=42)
  train.append(train_images)
  validation.append(validation_images)
  test.append(test_images)

In [11]:
#Make separate directories for each set
os.mkdir(os.path.join(projectDir, "train"))
os.mkdir(os.path.join(projectDir, "validation"))
os.mkdir(os.path.join(projectDir, "test"))

In [5]:
trainDir = projectDir + "/" + "train"
validationDir = projectDir + "/" + "validation"
testDir = projectDir + "/" + "test"

In [ ]:
#Insert data from each category into the newly created train, validation, and test folders
for i in range(len(categories)):
  category = categories[i]
  train_set = train[i]
  validation_set = validation[i]
  test_set = test[i]
  trainDestDir = os.path.join(trainDir, category)
  valDestDir = os.path.join(validationDir, category)
  testDestDir = os.path.join(testDir, category)
  os.mkdir(trainDestDir)
  os.mkdir(valDestDir)
  os.mkdir(testDestDir)

  for j in range(len(train_set)):
    impath = os.path.join(trainDestDir, f'image{j}.jpg')
    imageio.imwrite(impath, imageio.imread(train_set[j]))

  for j in range(len(validation_set)):
    impath = os.path.join(valDestDir, f'image{j}.jpg')
    imageio.imwrite(impath, imageio.imread(validation_set[j]))

  for j in range(len(test_set)):
    impath = os.path.join(testDestDir, f'image{j}.jpg')
    imageio.imwrite(impath, imageio.imread(test_set[j]))

In [6]:
#Build data generators
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        trainDir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        validationDir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.


In [ ]:
####### MODELING ########

In [1]:
'''We are going to try 4 different CNN architectures:
1)VGG16
2)ResNet50
3)InceptionV3
4)DenseNet201
'''

'We are going to try 3 different CNN architectures:\nVGG16\nResNet50\nInceptionV3\n'

In [7]:
#VGG16
from tensorflow.keras.applications import VGG16
vggModel = VGG16(include_top=False, weights='imagenet')
vggModel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [8]:
#Extract VGG16 Features - RUN ONLY ONCE AND THEN SAVE
vggTrainFeatures = vggModel.predict(train_generator)
vggValFeatures = vggModel.predict(validation_generator)

In [14]:
#Make a directory to save these features
cnnFeaturesDir = os.path.join(projectDir, 'CNN Architecture Features')
os.mkdir(cnnFeaturesDir)

In [15]:
#Save the features - it is somewhat expensive to recompute (~12 minutes)
np.save(cnnFeaturesDir + '/' + 'VGG16-train-features.npy', vggTrainFeatures)
np.save(cnnFeaturesDir + '/' + 'VGG16-val-features.npy', vggValFeatures)

In [17]:
#Retrieve features from project directory
vggTrainData = np.load(cnnFeaturesDir + '/' + 'VGG16-train-features.npy')
vggValData = np.load(cnnFeaturesDir + '/' + 'VGG16-val-features.npy')

In [18]:
from tensorflow.keras.utils import to_categorical
trainTarget = to_categorical(train_generator.labels)
valTarget = to_categorical(validation_generator.labels)

In [27]:
#Structure, compile and train the VGG16 model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, LeakyReLU

model1 = Sequential()
model1.add(Flatten(input_shape=vggTrainData.shape[1:]))
model1.add(Dense(200, activation=LeakyReLU(alpha=0.3)))
model1.add(Dropout(0.5)) 
model1.add(Dense(100, activation=LeakyReLU(alpha=0.3))) 
model1.add(Dropout(0.3)) 
model1.add(Dense(80, activation='softmax'))

model1.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

history = model1.fit(vggTrainData, trainTarget, epochs=75, batch_size=32, validation_data=(vggValData, valTarget))

(eval_loss, eval_accuracy) = model1.evaluate(vggValData, valTarget, batch_size=32, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

Epoch 1/75
80/80 [==============================] - 1s 5ms/step - loss: 4.7898 - accuracy: 0.0078 - val_loss: 4.4495 - val_accuracy: 0.0109
Epoch 2/75
80/80 [==============================] - 0s 4ms/step - loss: 4.4517 - accuracy: 0.0254 - val_loss: 4.4904 - val_accuracy: 0.0094
Epoch 3/75
80/80 [==============================] - 0s 4ms/step - loss: 4.2404 - accuracy: 0.0531 - val_loss: 4.5322 - val_accuracy: 0.0156
Epoch 4/75
80/80 [==============================] - 0s 4ms/step - loss: 4.0495 - accuracy: 0.0812 - val_loss: 4.5118 - val_accuracy: 0.0094
Epoch 5/75
80/80 [==============================] - 0s 4ms/step - loss: 3.7499 - accuracy: 0.1293 - val_loss: 4.7391 - val_accuracy: 0.0125
Epoch 6/75
80/80 [==============================] - 0s 4ms/step - loss: 3.4745 - accuracy: 0.1750 - val_loss: 4.8281 - val_accuracy: 0.0109
Epoch 7/75
80/80 [==============================] - 0s 4ms/step - loss: 3.1600 - accuracy: 0.2480 - val_loss: 4.8561 - val_accuracy: 0.0156
Epoch 8/75
80/80 [==